In [ ]:
#############################################################################
# zlib License
#
# (C) 2023 Murtaza Safdari <musafdar@cern.ch>
#
# This software is provided 'as-is', without any express or implied
# warranty.  In no event will the authors be held liable for any damages
# arising from the use of this software.
#
# Permission is granted to anyone to use this software for any purpose,
# including commercial applications, and to alter it and redistribute it
# freely, subject to the following restrictions:
#
# 1. The origin of this software must not be misrepresented; you must not
#    claim that you wrote the original software. If you use this software
#    in a product, an acknowledgment in the product documentation would be
#    appreciated but is not required.
# 2. Altered source versions must be plainly marked as such, and must not be
#    misrepresented as being the original software.
# 3. This notice may not be removed or altered from any source distribution.
#############################################################################

In [ ]:
#%%
%matplotlib inline
import matplotlib.pyplot as plt
import logging
import i2c_gui
import i2c_gui.chips
from i2c_gui.usb_iss_helper import USB_ISS_Helper
from i2c_gui.fpga_eth_helper import FPGA_ETH_Helper
import numpy as np
from mpl_toolkits.axes_grid1 import make_axes_locatable
# import time
from tqdm import tqdm
# from i2c_gui.chips.etroc2_chip import register_decoding
import os, sys
import multiprocessing
os.chdir(f'/home/{os.getlogin()}/ETROC2/ETROC_DAQ')
import run_script
import importlib
importlib.reload(run_script)
import datetime
import pandas
from pathlib import Path
import subprocess
import sqlite3
from notebooks.notebook_helpers import *
from fnmatch import fnmatch
import scipy.stats as stats
import hist
from hist import Hist
import mplhep as hep
plt.style.use(hep.style.CMS)
import boost_histogram as bh
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.collections import PolyCollection
from matplotlib.colors import colorConverter

In [ ]:
# !!!!!!!!!!!!
# It is very important to correctly set the chip name, this value is stored with the data
chip_names = ["ET2_CNM_1_3_HV210V_offset0x08","ET2_CNM_2_6_HV155V_offset0x08","ET2_EPIR_1_1_HV210V_offset0x08"]
chip_fignames = chip_names
chip_figtitles = ["ETROC2 BB CNM 1-3 HV210V OS:8","ETROC2 BB CNM 2-6 HV155V OS:8","ETROC2 BB EPIR 1-1 HV210V OS:8"]

# I2C addresses for the pixel block and WS
chip_addresses = [0x60,0x61,0x62]

ws_addresses = [None, None, None]

chip_labels= ["0","1","3"]

today = datetime.date.today().isoformat()
fig_outdir = Path('../ETROC-figures')
fig_outdir = fig_outdir / (today + '_Array_Test_Results')
fig_outdir.mkdir(exist_ok=True)
fig_path = str(fig_outdir)


In [ ]:
#%%
%matplotlib inline
import matplotlib.pyplot as plt

fig = plt.figure(dpi=50, figsize=(5,5))
gs = fig.add_gridspec(1,1)

ax0 = fig.add_subplot(gs[0,0])
ax0.plot([1, 0], [1, 0])
plt.show()

In [ ]:
h = {chip_name: hist.Hist(hist.axis.Regular(50, 150, 250, name="CAL", label="CAL [LSB]"), 
                          hist.axis.Regular(64, 0, 512,  name="TOT", label="TOT [LSB]"),
                          hist.axis.Regular(64, 0, 1024, name="TOA", label="TOA [LSB]"),
                          hist.axis.Regular(10, 0, 10,  name="numHits",label="Number of Hits in Event"),
                          hist.axis.Integer(0, 16, name="ROW", label="ROW"),
                          hist.axis.Integer(0, 16, name="COL", label="COL")
                          )
     for chip_name in chip_names}

In [ ]:
def return_hist():
    return {chip_name: hist.Hist(hist.axis.Regular(25, 140, 240, name="CAL", label="CAL [LSB]"), 
                          hist.axis.Regular(32, 0, 512,  name="TOT", label="TOT [LSB]"),
                          hist.axis.Regular(32, 0, 1024, name="TOA", label="TOA [LSB]"),
                          hist.axis.Regular(10, 0, 10,  name="numHits",label="Number of Hits in Event"),
                          hist.axis.Integer(0, 16, name="ROW", label="ROW"),
                          hist.axis.Integer(0, 16, name="COL", label="COL")
                          )
     for chip_name in chip_names}

In [ ]:
del h

In [ ]:
def fill_hist(chip_label, chip_name, hist, path_pattern=f"*{today}_Array_Test_Results/*", numHitsCut=-1, CALcut=(-1,-1), TOAcut=(-1,-1), TOTcut=(-1,-1), ROWcut=(-1,-1), COLcut=(-1,-1)):
    root = '../ETROC-Data'
    file_pattern = "*translated*.dat"
    file_list = []
    totalNumHits = 0
    for path, subdirs, files in os.walk(root):
        if not fnmatch(path, path_pattern): continue
        for name in files:
            pass
            if fnmatch(name, file_pattern):
                file_list.append(os.path.join(path, name))
    for file_name in (file_list): print(file_name)
    for file_name in tqdm(file_list):
        with open(file_name) as infile:
            miniList  = []
            writeList = False
            numHits = 0
            for line in infile:
                text_list = line.split()
                if(text_list[1]!=chip_label): continue
                if text_list[2]=="HEADER":
                    writeList = True
                if text_list[2]=="TRAILER":
                    numHits = int(text_list[8])
                    if len(miniList)!=numHits: pass
                    elif(numHitsCut>-1 and numHits>numHitsCut): pass
                    else:
                        totalNumHits += numHits
                        for selectedLine in miniList:
                            text_mini_list = selectedLine.split()
                            TOA = int(text_mini_list[10])
                            if(TOAcut[0]>-1 and TOAcut[1]>-1 and (TOA<TOAcut[0] or TOA>=TOAcut[1])): continue
                            TOT = int(text_mini_list[12])
                            if(TOTcut[0]>-1 and TOTcut[1]>-1 and (TOT<TOTcut[0] or TOT>=TOTcut[1])): continue
                            CAL = int(text_mini_list[14])
                            if(CALcut[0]>-1 and CALcut[1]>-1 and (CAL<CALcut[0] or CAL>=CALcut[1])): continue
                            ROW = int(text_mini_list[8])
                            if(ROWcut[0]>-1 and ROWcut[1]>-1 and (ROW<ROWcut[0] or ROW>=ROWcut[1])): continue
                            COL = int(text_mini_list[6])
                            if(COLcut[0]>-1 and COLcut[1]>-1 and (COL<COLcut[0] or COL>=COLcut[1])): continue
                            hist[chip_name].fill(CAL,TOT,TOA,numHits,ROW,COL)
                    miniList  = []
                    writeList = False
                    numHits = 0
                if(writeList and text_list[2]=="DATA"):
                    miniList.append(line)
    print("Total Number of Hits (After DF NHits cut)", totalNumHits)

In [ ]:
def make_inclusive_plots(chip_name, chip_figname, chip_figtitle, save=True, show=False, tag='', title_tag=''):
    
    fig = plt.figure(dpi=50, figsize=(20,20))
    gs = fig.add_gridspec(1,1)
    ax = fig.add_subplot(gs[0,0])
    ax.set_title(f"{chip_figtitle}, Heat Map{title_tag}", loc="right", size=25)
    hep.cms.text(loc=0, ax=ax, text="Preliminary", fontsize=25)
    h[chip_name].project("COL","ROW").plot2d(ax=ax, lw=1)
    ax.invert_xaxis()
    # ax.invert_yaxis()
    if(save): plt.savefig(fig_path+"/"+chip_figname+"_heatmap_"+tag+datetime.datetime.now().strftime("%Y-%m-%d_%H-%M")+".png")
    if(show): plt.show()
    plt.close()
    
    fig = plt.figure(dpi=50, figsize=(20,10))
    gs = fig.add_gridspec(1,1)
    ax = fig.add_subplot(gs[0,0])
    ax.set_title(f"{chip_figtitle}, numHits{title_tag}", loc="right", size=25)
    hep.cms.text(loc=0, ax=ax, text="Preliminary", fontsize=25)
    h[chip_name].project("numHits")[:10j].plot1d(ax=ax, lw=1)
    if(save): plt.savefig(fig_path+"/"+chip_figname+"_numHits_"+tag+datetime.datetime.now().strftime("%Y-%m-%d_%H-%M")+".png")
    if(show): plt.show()
    plt.close()

    fig = plt.figure(dpi=50, figsize=(20,10))
    gs = fig.add_gridspec(1,1)
    ax = fig.add_subplot(gs[0,0])
    ax.set_title(f"{chip_figtitle}, CAL{title_tag}", loc="right", size=25)
    hep.cms.text(loc=0, ax=ax, text="Preliminary", fontsize=25)
    h[chip_name].project("CAL")[:].plot1d(ax=ax, lw=1)
    if(save): plt.savefig(fig_path+"/"+chip_figname+"_CAL_"+tag+datetime.datetime.now().strftime("%Y-%m-%d_%H-%M")+".png")
    if(show): plt.show()
    plt.close()

    fig = plt.figure(dpi=50, figsize=(20,10))
    gs = fig.add_gridspec(1,1)
    ax = fig.add_subplot(gs[0,0])
    ax.set_title(f"{chip_figtitle}, TOT{title_tag}", loc="right", size=25)
    hep.cms.text(loc=0, ax=ax, text="Preliminary", fontsize=25)
    h[chip_name].project("TOT")[:].plot1d(ax=ax, lw=1)
    if(save): plt.savefig(fig_path+"/"+chip_figname+"_TOT_"+tag+datetime.datetime.now().strftime("%Y-%m-%d_%H-%M")+".png")
    if(show): plt.show()
    plt.close()

    fig = plt.figure(dpi=50, figsize=(20,10))
    gs = fig.add_gridspec(1,1)
    ax = fig.add_subplot(gs[0,0])
    ax.set_title(f"{chip_figtitle}, TOA{title_tag}", loc="right", size=25)
    hep.cms.text(loc=0, ax=ax, text="Preliminary", fontsize=25)
    h[chip_name].project("TOA")[:].plot1d(ax=ax, lw=1)
    if(save): plt.savefig(fig_path+"/"+chip_figname+"_TOA_"+tag+datetime.datetime.now().strftime("%Y-%m-%d_%H-%M")+".png")
    if(show): plt.show()
    plt.close()

    fig = plt.figure(dpi=50, figsize=(20,20))
    gs = fig.add_gridspec(1,1)
    ax = fig.add_subplot(gs[0,0])
    ax.set_title(f"{chip_figtitle}, TOA v TOT{title_tag}", loc="right", size=25)
    hep.cms.text(loc=0, ax=ax, text="Preliminary", fontsize=25)
    h[chip_name].project("TOA","TOT")[::2j,::2j].plot2d(ax=ax, lw=1)
    if(save): plt.savefig(fig_path+"/"+chip_figname+"_TOA_TOT_"+tag+datetime.datetime.now().strftime("%Y-%m-%d_%H-%M")+".png")
    if(show): plt.show()
    plt.close()

    fig = plt.figure(dpi=50, figsize=(20, 20))
    h[chip_name].project("TOA","TOT")[::2j,::2j].plot2d_full(
        top_ls="-",
        top_color="orange",
        top_lw=1,
        side_ls="-",
        side_lw=1,
        side_color="steelblue",
    )
    plt.tight_layout()
    if(save): plt.savefig(fig_path+"/"+chip_figname+"_TOA_TOT_full_"+tag+datetime.datetime.now().strftime("%Y-%m-%d_%H-%M")+".png")
    if(show): plt.show()
    plt.close()

    fig = plt.figure(dpi=50, figsize=(20, 20))
    h[chip_name].project("TOA","CAL")[::2j,::1j].plot2d_full(
        top_ls="-",
        top_color="orange",
        top_lw=1,
        side_ls="-",
        side_lw=1,
        side_color="steelblue",
    )
    plt.tight_layout()
    if(save): plt.savefig(fig_path+"/"+chip_figname+"_TOA_CAL_full_"+tag+datetime.datetime.now().strftime("%Y-%m-%d_%H-%M")+".png")
    if(show): plt.show()
    plt.close()

    fig = plt.figure(dpi=50, figsize=(20, 20))
    h[chip_name].project("TOT","CAL")[::2j,::1j].plot2d_full(
        top_ls="-",
        top_color="orange",
        top_lw=1,
        side_ls="-",
        side_lw=1,
        side_color="steelblue",
    )
    plt.tight_layout()
    if(save): plt.savefig(fig_path+"/"+chip_figname+"_TOT_CAL_full_"+tag+datetime.datetime.now().strftime("%Y-%m-%d_%H-%M")+".png")
    if(show): plt.show()
    plt.close()

In [ ]:
# "*2023-09-08_Array_Test_Results/Run*_ET2-CNM-Batch1-3_cosmic_1hr_run*"
# /home/daq/ETROC2/ETROC-Data/2023-09-14_Array_Test_Results/Run57_ET2_CNM_B1_3_HV210_ET2_CNM_B2_6_HV155_ET2_EPIR_B1_1_HV210_cosmic_run_offset_8

In [ ]:
h = return_hist()
fill_hist(chip_labels[0], chip_names[0], h, path_pattern=f"*2023-09-1[4-5]_Array_Test_Results/Run*_ET2_CNM_B1_3_HV210_ET2_CNM_B2_6_HV155_ET2_EPIR_B1_1_HV210_cosmic_run_offset_8", 
          numHitsCut=5, CALcut=(-1,-1), TOAcut=(-1,-1), TOTcut=(-1,-1), ROWcut=(-1,-1), COLcut=(-1,-1))
make_inclusive_plots(chip_names[0], chip_fignames[0], chip_figtitles[0], save=True, show=True, tag="inclusive_", title_tag=", Nhits<6")
del h

In [ ]:
h = return_hist()
fill_hist(chip_labels[2], chip_names[2], h, path_pattern=f"*2023-09-1[4-5]_Array_Test_Results/Run*_ET2_CNM_B1_3_HV210_ET2_CNM_B2_6_HV155_ET2_EPIR_B1_1_HV210_cosmic_run_offset_8", 
          numHitsCut=5, CALcut=(-1,-1), TOAcut=(-1,-1), TOTcut=(-1,-1), ROWcut=(-1,-1), COLcut=(-1,-1))
make_inclusive_plots(chip_names[2], chip_fignames[2], chip_figtitles[2], save=True, show=True, tag="inclusive_", title_tag=", NHits<6")
del h

In [ ]:
h = return_hist()
fill_hist(chip_labels[1], chip_names[1], h, path_pattern=f"*2023-09-1[4-5]_Array_Test_Results/Run*_ET2_CNM_B1_3_HV210_ET2_CNM_B2_6_HV155_ET2_EPIR_B1_1_HV210_cosmic_run_offset_8", 
          numHitsCut=5, CALcut=(-1,-1), TOAcut=(-1,-1), TOTcut=(-1,-1), ROWcut=(-1,-1), COLcut=(-1,-1))
make_inclusive_plots(chip_names[1], chip_fignames[1], chip_figtitles[1], save=False, show=True, tag="inclusive_", title_tag=", NHits<6")
del h

In [ ]:
def return_cosmic_3board_hist():
    return hist.Hist(hist.axis.Regular(10, 0, 5000, name="del_toa", label=fr"$t_2 - (t_0 + t_1)/2$ [ps]"),
                     hist.axis.Regular(7, 0, 20, name="del_bm", label=fr"Euclidean Distance bottom-middle hits [px coords]"),
                     hist.axis.Regular(7, 0, 20, name="del_mt", label=fr"Euclidean Distance middle-top hits [px coords]")
                     )

In [ ]:
def fill_cosmic_3board_hist(chip_labels, chip_names, hist, path_pattern=f"*{today}_Array_Test_Results/*", CALcut=(-1,-1), TOAcut=(-1,-1), TOTcut=(-1,-1), ROWcut=(-1,-1), COLcut=(-1,-1), event_displays=False, save=False):
    root = '../ETROC-Data'
    file_pattern = "*translated*.dat"
    file_list = []
    totalNumHits = 0
    totalMidHits = 0
    for path, subdirs, files in os.walk(root):
        if not fnmatch(path, path_pattern): continue
        for name in files:
            pass
            if fnmatch(name, file_pattern):
                file_list.append(os.path.join(path, name))
    for file_name in (file_list): print(file_name)
    for file_name in tqdm(file_list):
        with open(file_name) as infile:
            miniDict  = {cid:[] for cid in chip_labels}
            writeList = False
            numHits   = 0
            running_bcid = -1
            current_bcid = -1
            for line in infile:
                text_list = line.split()
                if(text_list[1] not in chip_labels): continue
                if text_list[2]=="HEADER":
                    current_bcid = text_list[-1]
                    if(current_bcid!=running_bcid and running_bcid!=-1): writeList=True
                    elif(current_bcid!=running_bcid and running_bcid==-1): running_bcid = current_bcid
                if text_list[2]=="TRAILER":
                    numHits += int(text_list[8])
                    if(text_list[1]==chip_labels[1] and int(text_list[8])<6): totalMidHits += int(text_list[8])
                if(text_list[2]=="DATA"):
                    miniDict[text_list[1]].append(line)
                    TOA = int(text_list[10])
                    TOT = int(text_list[12])
                    CAL = int(text_list[14])
                    ROW = int(text_list[8])
                    COL = int(text_list[6])
                    if( (ROWcut[0]>-1 and ROWcut[1]>-1 and (ROW<ROWcut[0] or ROW>=ROWcut[1])) or (COLcut[0]>-1 and COLcut[1]>-1 and (COL<COLcut[0] or COL>=COLcut[1])) or (TOAcut[0]>-1 and TOAcut[1]>-1 and (TOA<TOAcut[0] or TOA>=TOAcut[1])) or (TOTcut[0]>-1 and TOTcut[1]>-1 and (TOT<TOTcut[0] or TOT>=TOTcut[1])) or (CALcut[0]>-1 and CALcut[1]>-1 and (CAL<CALcut[0] or CAL>=CALcut[1])) ): 
                        writeList = True
                        numHits = -1
                if(writeList):
                    condition = True
                    for cid in chip_labels: condition = condition and (len(miniDict[cid])==1)
                    if(numHits==3 and condition):
                        totalNumHits += 1
                        if(event_displays):
                            xx, yy = np.meshgrid(np.arange(16), np.arange(16))
                            data = np.zeros((16,16,3))
                            for idx,cid in zip(range(3),chip_labels): 
                                for cline in miniDict[cid]:
                                    ctext = cline.split()
                                    TOA = int(ctext[10])
                                    TOT = int(ctext[12])
                                    CAL = int(ctext[14])
                                    ROW = int(ctext[8])
                                    COL = int(ctext[6])
                                    data[COL,ROW,idx] = 1
                            fig = plt.figure(dpi=50)
                            ax1 = fig.add_subplot(111, projection='3d')
                            ax1.grid(False)
                            ax1.contourf(xx, yy, data[:,:,0], 100, zdir='z', offset=0.0, alpha=0.15)
                            ax1.contourf(xx, yy, data[:,:,1], 100, zdir='z', offset=0.5, alpha=0.15, cmap="plasma")
                            ax1.contourf(xx, yy, data[:,:,2], 100, zdir='z', offset=1.0, alpha=0.15)
                            ax1.set_zlim((0.,1.0))
                            ax1.set_xlabel('COL')
                            ax1.set_ylabel('ROW')
                            ax1.invert_xaxis()
                            ax1.set_xticks(ticks=range(16),labels=[],minor=True)
                            ax1.set_yticks(ticks=range(16),labels=[],minor=True)
                            ax1.set_zticks(ticks=[0,0.5,1],labels=["B","M","T"])
                            ax1.tick_params(axis='x', labelsize=8)
                            ax1.tick_params(axis='y', labelsize=8)
                            ax1.tick_params(axis='z', labelsize=8)
                            ax1.grid(visible=False, axis='z')
                            ax1.grid(visible=True, which='major', axis='x')
                            ax1.grid(visible=True, which='major', axis='y')
                            plt.show()
                            del data,xx,yy

                    condition = True
                    for cid in chip_labels: condition = condition and (len(miniDict[cid])==1)
                    if(numHits==3 and condition):
                        toa_b = int(miniDict[chip_labels[0]][0].split()[10])*3125/int(miniDict[chip_labels[0]][0].split()[14])
                        toa_m = int(miniDict[chip_labels[1]][0].split()[10])*3125/int(miniDict[chip_labels[1]][0].split()[14])
                        toa_t = int(miniDict[chip_labels[2]][0].split()[10])*3125/int(miniDict[chip_labels[2]][0].split()[14])
                        row_b = int(miniDict[chip_labels[0]][0].split()[8])
                        row_m = int(miniDict[chip_labels[1]][0].split()[8])
                        row_t = int(miniDict[chip_labels[2]][0].split()[8])
                        col_b = int(miniDict[chip_labels[0]][0].split()[6])
                        col_m = int(miniDict[chip_labels[1]][0].split()[6])
                        col_t = int(miniDict[chip_labels[2]][0].split()[6])
                        hist.fill(np.abs(toa_m - ((toa_b+toa_t)/2)), np.sqrt((row_b-row_m)**2 + (col_b-col_m)**2), np.sqrt((row_m-row_t)**2 + (col_m-col_t)**2))
                    
                    writeList = False
                    running_bcid = -1
                    numHits = 0
                    del miniDict
                    miniDict  = {cid:[] for cid in chip_labels}

    print("Total Number of Events", totalNumHits)
    print("Total Number of Hits on Middle Board (<6 hits/event)", totalMidHits)

    fig = plt.figure(dpi=50, figsize=(20,10))
    gs = fig.add_gridspec(1,1)
    ax = fig.add_subplot(gs[0,0])
    # ax.set_title(f"", loc="right", size=25)
    hep.cms.text(loc=0, ax=ax, text="Preliminary", fontsize=25)
    hist.project("del_toa")[:].plot1d(ax=ax, lw=1)
    # if(save): plt.savefig(fig_path+"/"+chip_figname+"_TOA_"+tag+datetime.datetime.now().strftime("%Y-%m-%d_%H-%M")+".png")
    if(True): plt.show()
    plt.close()

    fig = plt.figure(dpi=50, figsize=(20,10))
    gs = fig.add_gridspec(1,1)
    ax = fig.add_subplot(gs[0,0])
    # ax.set_title(f"", loc="right", size=25)
    hep.cms.text(loc=0, ax=ax, text="Preliminary", fontsize=25)
    hist.project("del_bm")[:].plot1d(ax=ax, lw=1)
    # if(save): plt.savefig(fig_path+"/"+chip_figname+"_TOA_"+tag+datetime.datetime.now().strftime("%Y-%m-%d_%H-%M")+".png")
    if(True): plt.show()
    plt.close()

    fig = plt.figure(dpi=50, figsize=(20,10))
    gs = fig.add_gridspec(1,1)
    ax = fig.add_subplot(gs[0,0])
    # ax.set_title(f"", loc="right", size=25)
    hep.cms.text(loc=0, ax=ax, text="Preliminary", fontsize=25)
    hist.project("del_mt")[:].plot1d(ax=ax, lw=1)
    # if(save): plt.savefig(fig_path+"/"+chip_figname+"_TOA_"+tag+datetime.datetime.now().strftime("%Y-%m-%d_%H-%M")+".png")
    if(True): plt.show()
    plt.close()

In [ ]:
h = return_cosmic_3board_hist()
fill_cosmic_3board_hist(chip_labels, chip_names, h, path_pattern=f"*2023-09-1[4-5]_Array_Test_Results/Run*_ET2_CNM_B1_3_HV210_ET2_CNM_B2_6_HV155_ET2_EPIR_B1_1_HV210_cosmic_run_offset_8", 
          CALcut=(-1,-1), TOAcut=(-1,-1), TOTcut=(-1,-1), ROWcut=(-1,-1), COLcut=(-1,-1), event_displays=False, save=False)
# make_inclusive_plots(chip_names[0], chip_fignames[0], chip_figtitles[0], save=True, show=True, tag="inclusive_", title_tag=", Nhits<6")
del h

In [ ]:
fill_hist(chip_labels[0], chip_names[0], h, path_pattern=f"*2023-09-08_Array_Test_Results/Run*_ET2-CNM-Batch1-3_cosmic_1hr_run*", 
          numHitsCut=-1, CALcut=(-1,-1), TOAcut=(-1,-1), TOTcut=(-1,-1), ROWcut=(0,8), COLcut=(-1,-1))
make_inclusive_plots(chip_names[0], chip_fignames[0], chip_figtitles[0], save=False, show=True)
del h

In [ ]:
h = return_hist()
fill_hist(chip_labels[0], chip_names[0], h, path_pattern=f"*2023-09-08_Array_Test_Results/Run*_ET2-CNM-Batch1-3_cosmic_1hr_run*", 
          numHitsCut=-1, CALcut=(-1,-1), TOAcut=(-1,-1), TOTcut=(-1,-1), ROWcut=(0,8), COLcut=(0,8))
make_inclusive_plots(chip_names[0], chip_fignames[0], chip_figtitles[0], save=True, show=True, tag="bottomright_")
del h

In [ ]:
fill_hist(chip_labels[0], chip_names[0], h, path_pattern=f"*2023-09-08_Array_Test_Results/Run*_ET2-CNM-Batch1-3_cosmic_1hr_run*", 
          numHitsCut=-1, CALcut=(-1,-1), TOAcut=(-1,-1), TOTcut=(-1,-1), ROWcut=(0,8), COLcut=(8,16))
make_inclusive_plots(chip_names[0], chip_fignames[0], chip_figtitles[0], save=False, show=True)
del h

In [ ]:
fill_hist(chip_labels[0], chip_names[0], h, path_pattern=f"*2023-09-08_Array_Test_Results/Run*_ET2-CNM-Batch1-3_cosmic_1hr_run*", 
          numHitsCut=-1, CALcut=(-1,-1), TOAcut=(-1,-1), TOTcut=(-1,-1), ROWcut=(0,8), COLcut=(0,15))
make_inclusive_plots(chip_names[0], chip_fignames[0], chip_figtitles[0], save=False, show=True)
del h

In [ ]:
fill_hist(chip_labels[0], chip_names[0], h, path_pattern=f"*2023-09-08_Array_Test_Results/Run*_ET2-CNM-Batch1-3_cosmic_1hr_run*", 
          numHitsCut=-1, CALcut=(-1,-1), TOAcut=(-1,-1), TOTcut=(-1,-1), ROWcut=(-1,-1), COLcut=(0,8))
make_inclusive_plots(chip_names[0], chip_fignames[0], chip_figtitles[0], save=False, show=True)
del h

In [ ]:
fill_hist(chip_labels[0], chip_names[0], h, path_pattern=f"*2023-09-08_Array_Test_Results/Run*_ET2-CNM-Batch1-3_cosmic_1hr_run*", 
          numHitsCut=-1, CALcut=(-1,-1), TOAcut=(-1,-1), TOTcut=(-1,-1), ROWcut=(-1,-1), COLcut=(0,7))
make_inclusive_plots(chip_names[0], chip_fignames[0], chip_figtitles[0], save=False, show=True)
del h

In [ ]:
h = return_hist()
fill_hist(chip_labels[0], chip_names[0], h, path_pattern=f"*2023-09-08_Array_Test_Results/Run*_ET2-CNM-Batch1-3_cosmic_1hr_run*", 
          numHitsCut=-1, CALcut=(-1,-1), TOAcut=(-1,-1), TOTcut=(-1,-1), ROWcut=(0,6), COLcut=(0,15))
make_inclusive_plots(chip_names[0], chip_fignames[0], chip_figtitles[0], save=True, show=True, tag="bottom6minuscol15_")
del h

In [ ]:
def make_exclusive_plots(chip_name, chip_figname, chip_figtitle, save=True, show=False, selection=[(-1,-1),(-1,-1),(-1,-1),(-1,-1)], slice_dict={}):
    selection_key = ["CAL", "TOT", "TOA", "numHits"]
    selection_string = ""
    selected_hist = h[chip_name][slice_dict]
    for i in range(4):
        sel_0, sel_1 = selection[i]
        if(sel_0==-1 or sel_1==-1): continue
        selection_string += selection_key[i]+" "+f"{np.imag(sel_0)} to "+f"{np.imag(sel_1)}, "

    fig = plt.figure(dpi=50, figsize=(20,20))
    gs = fig.add_gridspec(1,1)
    ax = fig.add_subplot(gs[0,0])
    ax.set_title(f"{chip_figtitle}, Heat Map, Inclusive", loc="right", size=25)
    hep.cms.text(loc=0, ax=ax, text="Preliminary", fontsize=25)
    selected_hist.project("COL","ROW").plot2d(ax=ax, lw=1)
    ax.invert_xaxis()
    # ax.invert_yaxis()
    if(save): plt.savefig(fig_path+"/"+chip_figname+"_heatmap_exc_"+datetime.datetime.now().strftime("%Y-%m-%d_%H-%M")+".png")
    if(show): plt.show()
    plt.close()

    fig = plt.figure(dpi=100, figsize=(20,10))
    gs = fig.add_gridspec(1,1)
    ax = fig.add_subplot(gs[0,0])
    ax.set_title(f"{chip_figtitle}, numHits, {selection_string}", size=15, loc="right")
    hep.cms.text(loc=0, ax=ax, text="Preliminary", fontsize=25)
    selected_hist.project("numHits").plot1d(ax=ax, lw=1)
    if(save): plt.savefig(fig_path+"/"+chip_figname+"_numHits_exc_"+datetime.datetime.now().strftime("%Y-%m-%d_%H-%M")+".png")
    if(show): plt.show()
    plt.close()

    fig = plt.figure(dpi=100, figsize=(20,10))
    gs = fig.add_gridspec(1,1)
    ax = fig.add_subplot(gs[0,0])
    ax.set_title(f"{chip_figtitle}, CAL, {selection_string}", size=15, loc="right")
    hep.cms.text(loc=0, ax=ax, text="Preliminary", fontsize=25)
    selected_hist.project("CAL").plot1d(ax=ax, lw=1)
    if(save): plt.savefig(fig_path+"/"+chip_figname+"_CAL_exc_"+datetime.datetime.now().strftime("%Y-%m-%d_%H-%M")+".png")
    if(show): plt.show()
    plt.close()

    fig = plt.figure(dpi=100, figsize=(20,10))
    gs = fig.add_gridspec(1,1)
    ax = fig.add_subplot(gs[0,0])
    ax.set_title(f"{chip_figtitle}, TOT, {selection_string}", size=15, loc="right")
    hep.cms.text(loc=0, ax=ax, text="Preliminary", fontsize=25)
    selected_hist.project("TOT").plot1d(ax=ax, lw=1)
    if(save): plt.savefig(fig_path+"/"+chip_figname+"_TOT_exc_"+datetime.datetime.now().strftime("%Y-%m-%d_%H-%M")+".png")
    if(show): plt.show()
    plt.close()

    fig = plt.figure(dpi=100, figsize=(20,10))
    gs = fig.add_gridspec(1,1)
    ax = fig.add_subplot(gs[0,0])
    ax.set_title(f"{chip_figtitle}, TOA, {selection_string}", size=15, loc="right")
    hep.cms.text(loc=0, ax=ax, text="Preliminary", fontsize=25)
    selected_hist.project("TOA").plot1d(ax=ax, lw=1)
    if(save): plt.savefig(fig_path+"/"+chip_figname+"_TOA_exc_"+datetime.datetime.now().strftime("%Y-%m-%d_%H-%M")+".png")
    if(show): plt.show()
    plt.close()

    fig = plt.figure(dpi=100, figsize=(20,20))
    gs = fig.add_gridspec(1,1)
    ax = fig.add_subplot(gs[0,0])
    ax.set_title(f"{chip_figtitle}, TOA v TOT, {selection_string}", size=15, loc="right")
    hep.cms.text(loc=0, ax=ax, text="Preliminary", fontsize=25)
    selected_hist.project("TOA","TOT").plot2d(ax=ax, lw=1)
    if(save): plt.savefig(fig_path+"/"+chip_figname+"_TOA_TOT_exc_"+datetime.datetime.now().strftime("%Y-%m-%d_%H-%M")+".png")
    if(show): plt.show()
    plt.close()

    fig = plt.figure(dpi=100, figsize=(20, 20))
    selected_hist.project("TOA","TOT")[::2j,::2j].plot2d_full(
        top_ls="-",
        top_color="orange",
        top_lw=1,
        side_ls="-",
        side_lw=1,
        side_color="steelblue",
    )
    plt.tight_layout()
    if(save): plt.savefig(fig_path+"/"+chip_figname+"_TOA_TOT_full_exc_"+datetime.datetime.now().strftime("%Y-%m-%d_%H-%M")+".png")
    if(show): plt.show()
    plt.close()

In [ ]:
s1, s2, s3, s4, s5 = bh.tag.Slicer(),bh.tag.Slicer(),bh.tag.Slicer(),bh.tag.Slicer(),bh.tag.Slicer()

make_exclusive_plots(chip_names[0], chip_fignames[0], chip_figtitles[0], save=False, show=True, selection=[(150j,200j),(0j,450j),(100j,500j),(0j,2j)], slice_dict={"CAL":s1[150j:200j], "TOT":s2[0j:450j], "TOA":s3[100j:500j], "numHits":s4[0j:2j], "ROW":s5[0j:8j]})

In [ ]:
s1, s2, s3, s4, s5 = bh.tag.Slicer(),bh.tag.Slicer(),bh.tag.Slicer(),bh.tag.Slicer(),bh.tag.Slicer()

make_exclusive_plots(chip_names[0], chip_fignames[0], chip_figtitles[0], save=False, show=True, slice_dict={"ROW":s5[0j:8j]})

In [ ]:
s1, s2, s3, s4, s5 = bh.tag.Slicer(),bh.tag.Slicer(),bh.tag.Slicer(),bh.tag.Slicer(),bh.tag.Slicer()

make_exclusive_plots(chip_names[0], chip_fignames[0], chip_figtitles[0], save=False, show=True, slice_dict={"ROW":s5[8j:16j]})

In [ ]:
del h